Script must be run in same directory as data source (i.e., Excel spreadsheet of map labels).

In [1]:
# Define variables; then, run script
filename_of_input_labels_spreadsheet = 'labels.xlsx' # Include file extension
name_of_field_for_call_number = 'call_number'
name_of_field_for_description = 'description'
name_of_field_for_sheet_range = 'sheet_range'
name_of_field_for_scale = 'scale'
desired_name_for_combined_sheet_range_and_scale_field = 'sheet_range_and_scale'
filename_of_output_labels_spreadsheet = 'labels_manipulated.xlsx' # Include file extension

In [2]:
# Import required libraries
import pandas as pd
import xlsxwriter

In [3]:
# Read labels Excel file into a pandas DataFrame
df = pd.read_excel(filename_of_input_labels_spreadsheet)

In [4]:
# Combine values of 'sheet_range' and 'scale' fields, using return to separate values on different lines
# df[desired_name_for_combined_sheet_range_and_scale_field] = df[name_of_field_for_sheet_range].str.cat(df[name_of_field_for_scale], sep = '\n', na_rep = '')
df[desired_name_for_combined_sheet_range_and_scale_field] = df[[name_of_field_for_sheet_range, name_of_field_for_scale]].apply(lambda x: '\n'.join(x.dropna()), axis = 1)

In [5]:
# Drop 'sheet_range' and 'scale' fields, as they are no longer needed
df = df.drop(columns = [name_of_field_for_sheet_range, name_of_field_for_scale])

In [6]:
# Increment each index value by one
df.index = [i + 1 for i in df.index]

In [7]:
# Create empty DataFrame
empty_df = pd.DataFrame(
    index = range(0, len(df)),
    columns = ['left_spacer', 'middle_left_spacer', 'middle_right_spacer', 'right_spacer'], # 'Spacer' columns within Excel file
    data = ''
)

In [8]:
# Create second empty DataFrame
empty_df2 = pd.DataFrame(
    index = range(1, len(df) + 1),
    columns = ['left_spacer', 'middle_left_spacer', 'middle_right_spacer', 'right_spacer'], # 'Spacer' columns within Excel file
    data = ''
)

In [9]:
# Concatenate df, empty_df, and empty_df2
combined_df = pd.concat([df, empty_df, empty_df2])

In [10]:
# Sort combined_df to intersperse empty rows from empty_df
combined_df = combined_df.sort_index(kind = 'stable')

In [11]:
# # Add 'helper' column to assist with sorting in Excel
# combined_df['helper_column'] = ['header_spacer'] + (['text', 'horizontal_spacer_bottom', 'horizontal_spacer_top'] * (int(len(combined_df) / 3) - 1)) + ['text', 'horizontal_spacer_bottom']

In [12]:
# Reorder fields
combined_df = combined_df[[
    'left_spacer',
    name_of_field_for_call_number,
    'middle_left_spacer',
    name_of_field_for_description,
    'middle_right_spacer',
    desired_name_for_combined_sheet_range_and_scale_field,
    'right_spacer',
    # 'helper_column'
]]

#### Write resulting DataFrame back to Excel file

In [13]:
# Create a pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(filename_of_output_labels_spreadsheet, engine = 'xlsxwriter')

In [14]:
# Convert the dataframe to an XlsxWriter Excel object
combined_df.to_excel(writer, sheet_name = 'map_drawer_labels', index = False, header = False)

In [15]:
# Get the XlsxWriter workbook and worksheet objects
workbook  = writer.book
worksheet = writer.sheets['map_drawer_labels']

In [16]:
# workbook = xlsxwriter.Workbook(filename_of_output_labels_spreadsheet)
# worksheet = workbook.add_worksheet()

In [17]:
# Configure cell formatting
cells_without_borders = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Calibri',
    'font_size': 16,
    'font_color': 'black',
    'text_wrap': True,
    'shrink': True
})

In [18]:
bottom_border = workbook.add_format({
    # 'align': 'center',
    # 'valign': 'vcenter',
    # 'font_name': 'Calibri',
    # 'font_size': 16,
    # 'font_color': 'black',
    # 'text_wrap': True,
    # 'shrink': True,
    'bottom': 3
})

In [19]:
right_border = workbook.add_format({
    # 'align': 'center',
    # 'valign': 'vcenter',
    # 'font_name': 'Calibri',
    # 'font_size': 16,
    # 'font_color': 'black',
    # 'text_wrap': True,
    # 'shrink': True,
    'right': 3
})

In [20]:
right_and_bottom_border = workbook.add_format({
    'align': 'center',
    'valign': 'vcenter',
    'font_name': 'Calibri',
    'font_size': 16,
    'font_color': 'black',
    'text_wrap': True,
    'shrink': True,
    'right': 3,
    'bottom': 3
})

In [21]:
# Set row heights for each label
# Width is second argument in .set_row() method and is in character units
worksheet.set_row(0, 10)

for row in range(1, len(combined_df) + 1, 3):
    worksheet.set_row(row, 50, cells_without_borders)

for row in range(2, len(combined_df) + 1, 3):
    worksheet.set_row(row, 20, bottom_border)

for row in range(3, len(combined_df), 3):
    worksheet.set_row(row, 20)

In [22]:
# Set column widths for each label
# Width is third argument and is in character units
worksheet.set_column(0, 0, 2) # Column 'A'
worksheet.set_column(1, 1, 15) # Column 'B'
worksheet.set_column(2, 2, 2) # Column 'C'
worksheet.set_column(3, 3, 20) # Column 'D'
worksheet.set_column(4, 4, 2) # Column 'E'
worksheet.set_column(5, 5, 15) # Column 'F'
worksheet.set_column(6, 6, 4, right_border) # Column 'G'

0

In [23]:
# # Apply formatting and bottom and right borders to each label
# for i in range(1, len(combined_df), 3):
#     worksheet.conditional_format(i, 1, i, 5, {'type': 'cell',
#                                               'criteria': '!=',
#                                               'value': 9999999999,
#                                               'format': cells_without_borders})

In [24]:
# # Apply formatting to top row of cells without borders for each label
# for i in range(0, len(combined_df) + 1, 3):
#     worksheet.conditional_format(i, i, 0, 5, {'type': 'cell',
#                                               'criteria': '!=',
#                                               'value': '"It is not likely the value will match this string!"',
#                                               'format': cells_without_borders})

In [25]:
# # Apply formatting to middle row of cells without borders for each label
# for i in range(1, len(combined_df) + 1, 3):
#     worksheet.conditional_format(i, i, 0, 5, {'type': 'cell',
#                                               'criteria': '!=',
#                                               'value': '"It is not likely the value will match this string!"',
#                                               'format': cells_without_borders})

In [26]:
# # Add bottom border for each label
# for i in range(2, len(combined_df) + 1, 3):
#     worksheet.conditional_format(i, i, 0, 5, {'type': 'cell',
#                                               'criteria': '!=',
#                                               'value': '"It is not likely the value will match this string!"',
#                                               'format': bottom_border})

In [27]:
# # Apply formatting and border to top row in last column for each label
# for i in range(0, len(combined_df) + 1, 3):
#     worksheet.conditional_format(i, i, 6, 6, {'type': 'cell',
#                                               'criteria': '!=',
#                                               'value': '"It is not likely the value will match this string!"',
#                                               'format': right_border})

In [28]:
# # Apply formatting and border to middle row in last column for each label
# for i in range(1, len(combined_df) + 1, 3):
#     worksheet.conditional_format(i, i, 6, 6, {'type': 'cell',
#                                               'criteria': '!=',
#                                               'value': '"It is not likely the value will match this string!"',
#                                               'format': right_border})

In [29]:
# # Add right and bottom border in bottom-right cell for each label
# for i in range(2, len(combined_df) + 1, 3):
#     worksheet.conditional_format(i, i, 6, 6, {'type': 'cell',
#                                               'criteria': '!=',
#                                               'value': '"It is not likely the value will match this string!"',
#                                               'format': right_and_bottom_border})

In [30]:
# # Add the format to column 2 (zero-indexed) and adjust the width
# worksheet.set_column(2, 2, 15, text_wrap_format)

In [31]:
# Set paper size to 11 in. x 17 in.
worksheet.set_paper(3)

In [32]:
# Set page breaks
worksheet.set_h_pagebreaks(range(30, len(combined_df), 30))

In [33]:
# Set margins to zero
worksheet.set_margins(
    left = 0,
    right = 0,
    top = 0,
    bottom = 0
)

In [34]:
# Set the cells to be printed
worksheet.print_area(0, 0, len(combined_df) + 1, 6)

0

In [35]:
# Close the pandas Excel writer, and output the Excel file
writer.save()

C:\Users\badimond\AppData\Local\Temp\ipykernel_8528\2045151010.py:2: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
